In [ ]:
import torch
import numpy as np
import pandas as pd

In [ ]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
THRESHOLD = 0.5

In [ ]:
class EmbeddingClassifier(nn.Module):
    def __init__(self, emb_dim, num_classes, hidden_dims=[512,256], dropout=0.3):
        super().__init__()
        self.fc1 = nn.Linear(emb_dim, hidden_dims[0])
        self.bn1 = nn.BatchNorm1d(hidden_dims[0])
        self.drop1 = nn.Dropout(dropout)
        self.fc2 = nn.Linear(hidden_dims[0], hidden_dims[1])
        self.bn2 = nn.BatchNorm1d(hidden_dims[1])
        self.drop2 = nn.Dropout(dropout)
        self.out = nn.Linear(hidden_dims[1], num_classes)

    def forward(self, x):
        x = F.relu(self.bn1(self.fc1(x)))
        x = self.drop1(x)
        x = F.relu(self.bn2(self.fc2(x)))
        x = self.drop2(x)
        return self.out(x)

In [ ]:
test_df = pd.read_csv("/home/jovyan/Features/manifest_test.csv")
sample = test_df.sample(1).iloc[0]  # random pick, or use .iloc[n]
print("Running inference on chunk:", sample.chunk_id)

In [ ]:
train_meta = pd.read_csv("/home/jovyan/Data/birdclef-2025/train.csv", usecols=["filename","secondary_labels"])
train_meta["recording_id"] = train_meta.filename.str.replace(r"\.ogg$","",regex=True)
train_meta["sec_list"] = train_meta.secondary_labels.fillna("").str.split()
# gather all labels seen at train time
labels = set()
for _, r in test_df.iterrows():
    rid = r.chunk_id.split("_chk")[0]
    labels.add(r.primary_label)
    labels.update(train_meta.loc[train_meta.recording_id==rid, "sec_list"].values[0])
classes = sorted(labels)

In [ ]:
emb_arr = np.load(sample.emb_path)["embedding"]
emb_dim = emb_arr.shape[1]

model = EmbeddingClassifier(
    emb_dim=emb_dim,
    num_classes=len(classes),
    hidden_dims=[512,256],
    dropout=0.3
).to(DEVICE)

ckpt = torch.load("best_ckpt.pt", map_location=DEVICE)
model.load_state_dict(ckpt["model_state"])
model.eval()

In [ ]:
sample_emb = emb_arr.mean(axis=0).astype(np.float32)
x = torch.from_numpy(sample_emb).unsqueeze(0).to(DEVICE)  # shape [1, emb_dim]

with torch.no_grad():
    logits = model(x)                   # [1, num_classes]
    probs = torch.sigmoid(logits)[0]    # [num_classes]

In [ ]:
pred_idxs = (probs >= THRESHOLD).nonzero(as_tuple=False).squeeze().tolist()
if isinstance(pred_idxs, int):
    pred_idxs = [pred_idxs]

pred_labels = [classes[i] for i in pred_idxs]
pred_scores = [float(probs[i]) for i in pred_idxs]

print("\nPredicted species (threshold ≥ {:.2f}):".format(THRESHOLD))
for lab, score in zip(pred_labels, pred_scores):
    print(f"  • {lab}: {score:.3f}")